In [1]:
import telebot
import sqlite3
from telebot import types
BD_NAME = "bro_example.db"

conn = sqlite3.connect(BD_NAME)
cursor = conn.cursor()
bot = telebot.TeleBot(":)")


In [ ]:
query = "CREATE TABLE \"planner\" (\"ID\" INTEGER UNIQUE, \"user_id\" INTEGER, \"plan\" TEXT, PRIMARY KEY (\"ID\"))"
cursor.execute(query)


In [ ]:
#cursor.execute('INSERT INTO planner (user_id, plan) VALUES (1, "сделать питончик")')

#cursor.execute('SELECT * FROM planner)
# conn.commit()
cursor.execute('SELECT * FROM planner')
cursor.fetchall()

In [2]:
@bot.message_handler(commands=['bro'])
def send_keyboard(message, text="Привет, чем я могу тебе помочь?"):
    keyboard = types.ReplyKeyboardMarkup(row_width=1)  # наша клавиатура
    itembtn1 = types.KeyboardButton('Добваить задание') # создадим кнопку
    itembtn2 = types.KeyboardButton('Показать все задания') # создадим кнопку
    itembtn3 = types.KeyboardButton('Уничтожить задание')
    keyboard.add(itembtn1, itembtn2, itembtn3) # добавим кнопки 1 и 2 на первый ряд
    msg = bot.send_message(message.from_user.id, text="Привет медвед", reply_markup=keyboard)
    bot.register_next_step_handler(msg, callback_worker)

In [3]:
# напишем функции для каждого случая
# эта добавляет строчку с планом в хранилище

def add_plan(msg):
#     with sqlite3.connect(BD_NAME) as con:
#         cursor = con.cursor()
#         cursor.execute(f'INSERT INTO planner (user_id, plan) VALUES ({msg.from_user.id}, "{msg.text}")')
#         con.commit()
    bot.send_message(msg.chat.id, 'Добвил бро')
    send_keyboard(msg, "Чем еще могу помочь?")

In [14]:
'DELETE FROM planner WHERE user_id==? AND plan==?'
def delete_plan(msg):
    with sqlite3.connect(BD_NAME) as con:
        cursor = con.cursor()
        cursor.execute(f'DELETE FROM planner WHERE user_id == {msg.from_user.id} AND plan == "{msg.text}"')
        con.commit()
    bot.send_message(msg.chat.id, 'Уничтожил бро')
    send_keyboard(msg, "Чем еще могу помочь?")

In [15]:
def callback_worker(call):
    if call.text == 'Добваить задание':
        msg = bot.send_message(call.chat.id, 'какое задание?')
        bot.register_next_step_handler(msg, add_plan)
    elif call.text == 'Показать все задания':
        all_plans = None
        with sqlite3.connect(BD_NAME) as con:
            cursor = con.cursor()
            cursor.execute(f'SELECT * FROM planner WHERE user_id == {call.chat.id}')
            all_plans = cursor.fetchall()
        answer = ''
        for plan_id, user_id, plan_text in all_plans:
            answer += plan_text + '\n'
        bot.send_message(call.chat.id, answer)
    elif call.text == 'Уничтожить задание':
        msg = bot.send_message(call.chat.id, 'какое задание?')
        bot.register_next_step_handler(msg, delete_plan)
    else:
        bot.send_message(call.chat.id, 'Wat??')

In [16]:
bot.polling(none_stop=True)